# 🤝 Interactions

Before you start:
- Make sure you have the following packages installed via `pip`:
    - TODO ~~`energyplus-ooep`~~
    - `energyplus-dataset`

What you will learn in this chapter:
- How to _control_ simulator lifecycles.
- How to _access_ simulator variables
- How to _subscribe_ to simulator events.
- How to _visualize_ simulator variables.

TODO

## 🧳 Prepare

In [1]:
from controllables.core import TemporaryUnavailableError
from controllables.energyplus import World, Actuator, OutputMeter, OutputVariable

from energyplus.dataset.basic import dataset as _epds_

## 💨 Run!

In [2]:
world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / '1ZoneEvapCooler.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    # TODO
    #runtime=dict(recurring=True),
)

In [3]:
_ = world.add('logging:progress').start()
#world.run()

  0%|          | 0/100 [00:00<?, ?it/s]

In [4]:
#world.stop()

## 👀 Observe

TODO introduce manager pattern; references

### 🔣 Variables

In [5]:
list(world.variables.keys())

[]

### 🔔 Events

In [6]:
list(world.events.keys())

[Ref(name='message', include_warmup=True),
 Ref(name='progress', include_warmup=True)]

Keep me updated - Get [in the "loop"](https://wiktionary.org/wiki/in_the_loop), [literally](https://wikipedia.org/wiki/Event_loop).

In [7]:
_ = '''
world.events.on('message', lambda event: print('🥳', event))

@world.on('message')
def _(event): print('😋', event)

@world.events['message'].on
def _(event): print('😎', event)
'''

### 💲 Data

TODO

In [9]:
try:
    print(world['wallclock'].value)
    print(
        world[OutputVariable.Ref(
            type='Site Outdoor Air Drybulb Temperature', 
            key='ENVIRONMENT',
        )].value
    )
    print(
        world.variables[
            Actuator.Ref(
                type='Weather Data', 
                control_type='Environment', 
                key='Outdoor Dry Bulb',
            )
        ].value        
    )
except TemporaryUnavailableError:
    pass

2005-01-10 08:20:00
2.833333333333333


/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/energyplus/variables.py:365: RuntimeWarning: <controllables.energyplus.variables.OutputVariable object at 0x7f5e6c31fed0> requested while <controllables.energyplus._core.Core object at 0x7f5e6c337f90> is running; It may not be available until the next run
  _warnings_.warn(


In [10]:
from controllables.core.tools.records import VariableRecords

records = VariableRecords({
    '🕰️': world['wallclock:calendar'],
    'Δ🌡️': 
        world[OutputVariable.Ref('Site Outdoor Air Drybulb Temperature', 'ENVIRONMENT')] 
        - world[OutputVariable.Ref('Zone Mean Air Temperature', 'MAIN ZONE')],
}).watch(world.events['step'])

/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/energyplus/variables.py:365: RuntimeWarning: <controllables.energyplus.variables.OutputVariable object at 0x7f5e6c1b7d90> requested while <controllables.energyplus._core.Core object at 0x7f5e6c337f90> is running; It may not be available until the next run
  _warnings_.warn(


In [11]:
records.plot.scatter(x='🕰️', y='Δ🌡️').watch(records.events['change'] % 100)

## 🤔 Reflect

### 🎩 Tricks

In [12]:
world.variables.available_keys()

Exception ignored on calling ctypes callback function: <function EventManager._core_callback_setters.<locals>._Dispatcher._state at 0x7f5e6c1da840>
Traceback (most recent call last):
  File "/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 143, in cb_
    raise e
  File "/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 140, in cb_
    return cb(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 167, in _state
    self._event.__call__(
  File "/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/energyplus/events.py", line 105, in __call__
    return super().__call__(context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/AD/user/lab/EnergyPlus-OOEP/packages/controllables/core/callbacks.py", line 413, in __call__
    return self._callables.__call__(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  